In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("bourret2012/Genotypes_Dryad_without_Foyle_river.txt", header=0, index_col=None, sep="\t")

In [3]:
df.head()

,Pop_ID,Sample_ID,GCR_cBin24032_Ctg1_76,GCR_cBin13749_Ctg1_18,ESTNV_28684_665,GCR_cBin32777_Ctg1_66,GCR_cBin13749_Ctg1_171,GCR_cBin22083_Ctg1_38,GCR_cBin7800_Ctg1_739,ESTV_16768_432,...,ESTNV_32436_2335,ESTNV_36393_807,ESTV_18069_370,GCR_cBin8786_Ctg1_262,ESTNV_36510_747,ESTNV_37554_1370,ESTNV_33773_1438,ESTNV_20585_61,ESTNV_24373_965,ESTV_21166_231
0,Tuloma,01_RUS_TUL_01,A/T,A/A,A/A,C/C,A/A,A/C,A/A,A/A,...,G/G,G/G,G/G,A/A,G/G,G/G,A/A,A/A,G/G,G/G
1,Tuloma,01_RUS_TUL_02,-/-,A/C,-/-,G/G,A/A,A/A,A/T,A/A,...,G/G,G/G,G/G,A/A,G/G,G/G,A/A,A/A,G/G,G/G
2,Tuloma,01_RUS_TUL_03,T/T,A/C,A/A,C/G,A/A,A/A,T/T,A/T,...,G/G,G/G,G/G,A/A,G/G,G/G,A/A,A/A,G/G,G/G
3,Tuloma,01_RUS_TUL_04,T/T,A/A,A/A,C/C,A/G,C/C,A/T,A/A,...,G/G,G/G,G/G,A/A,G/G,G/G,A/A,A/A,G/G,G/G
4,Tuloma,01_RUS_TUL_05,A/T,A/A,A/A,C/C,A/A,A/A,A/T,A/A,...,G/G,G/G,G/G,A/A,G/G,G/G,A/A,A/A,G/G,G/G


In [4]:
landlock_group = ['RUS_PIS_LL', 'RUS_SYS_LL', 'RUS_PYA_LL', 'NOR_NAM_LL', 'NOR_BYG_LL', 'BristolCove']
anadormous_group = list(set(pd.unique(df.Pop_ID)) - set(landlock_group))

In [5]:
len(df[df.Pop_ID.isin(landlock_group)])

232

In [6]:
len(df[df.Pop_ID.isin(anadormous_group)])

1063

In [7]:
df = df.replace('-/-', -9)

In [8]:
Class_col = df.Pop_ID.isin(anadormous_group).map(lambda x: int(x)).rename("Class")

In [9]:
feature_table = pd.concat([df.Sample_ID, Class_col], axis=1)

In [10]:
def biallele2code(ref, ref_biallele, biallele):
    if biallele == -9:
        return -9
    
    if biallele == ref_biallele:
        return 0
    elif ref in biallele:
        return 1
    else:
        return 2

col_list = [df.Sample_ID, Class_col]
for i in range(2, len(df.columns)):
    genotype_cnt = df[df.columns[i]].value_counts()
    
    nucl_dict = {}
    for j in genotype_cnt.index:
        if j == -9:
            continue
        nucl = j.split("/")
        nucl_dict.setdefault(nucl[0], 0)
        nucl_dict.setdefault(nucl[1], 0)
        nucl_dict[nucl[0]] += genotype_cnt[j]
        nucl_dict[nucl[1]] += genotype_cnt[j]
    
    ref_nucl = list(nucl_dict.keys())[0]
    for nucl in nucl_dict.keys():
        if nucl_dict[nucl] > nucl_dict[ref_nucl]:
            ref_nucl = nucl
    
    ref_biallele = "%s/%s" % (ref_nucl, ref_nucl)
    
    coded_biallele = df[df.columns[i]].map(lambda x : biallele2code(ref_nucl, ref_biallele, x))
    
    col_list.append(coded_biallele)

In [11]:
feature_table = pd.concat(col_list, axis=1)

In [12]:
feature_table.head(10)

,Sample_ID,Class,GCR_cBin24032_Ctg1_76,GCR_cBin13749_Ctg1_18,ESTNV_28684_665,GCR_cBin32777_Ctg1_66,GCR_cBin13749_Ctg1_171,GCR_cBin22083_Ctg1_38,GCR_cBin7800_Ctg1_739,ESTV_16768_432,...,ESTNV_32436_2335,ESTNV_36393_807,ESTV_18069_370,GCR_cBin8786_Ctg1_262,ESTNV_36510_747,ESTNV_37554_1370,ESTNV_33773_1438,ESTNV_20585_61,ESTNV_24373_965,ESTV_21166_231
0,01_RUS_TUL_01,1,1,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,01_RUS_TUL_02,1,-9,1,-9,2,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,01_RUS_TUL_03,1,0,1,0,1,0,0,2,1,...,0,0,0,0,0,0,0,0,0,0
3,01_RUS_TUL_04,1,0,2,0,0,1,2,1,0,...,0,0,0,0,0,0,0,0,0,0
4,01_RUS_TUL_05,1,1,2,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
5,01_RUS_TUL_06,1,-9,1,1,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
6,01_RUS_TUL_07,1,0,1,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
7,01_RUS_TUL_08,1,1,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
8,01_RUS_TUL_09,1,0,2,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
9,01_RUS_TUL_10,1,0,2,0,0,0,2,0,1,...,0,0,0,0,0,0,0,0,0,0


In [15]:
feature_table.Class.value_counts()

1    1063
0     232
Name: Class, dtype: int64

In [13]:
feature_table.to_csv("bourret2012.tsv", sep="\t", header=True, index=False)